<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dependencies" data-toc-modified-id="Dependencies-1">Dependencies</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2">Functions</a></span></li><li><span><a href="#Paths" data-toc-modified-id="Paths-3">Paths</a></span></li><li><span><a href="#Main" data-toc-modified-id="Main-4">Main</a></span><ul class="toc-item"><li><span><a href="#Cleanup-INT-data" data-toc-modified-id="Cleanup-INT-data-4.1">Cleanup INT data</a></span></li><li><span><a href="#Cleanup-VDJdb-file---curated-by-Shirin" data-toc-modified-id="Cleanup-VDJdb-file---curated-by-Shirin-4.2">Cleanup VDJdb file - curated by Shirin</a></span></li><li><span><a href="#Cleanup-Adaptive-data" data-toc-modified-id="Cleanup-Adaptive-data-4.3">Cleanup Adaptive data</a></span><ul class="toc-item"><li><span><a href="#compile-all-adaptive-files-together-and-cleanup" data-toc-modified-id="compile-all-adaptive-files-together-and-cleanup-4.3.1">compile all adaptive files together and cleanup</a></span></li></ul></li></ul></li><li><span><a href="#bind-all" data-toc-modified-id="bind-all-5">bind all</a></span></li><li><span><a href="#Backup" data-toc-modified-id="Backup-6">Backup</a></span><ul class="toc-item"><li><span><a href="#Cleanup-Adaptive-data" data-toc-modified-id="Cleanup-Adaptive-data-6.1">Cleanup Adaptive data</a></span></li></ul></li></ul></div>

# Dependencies

In [1]:
pacman::p_load(data.table)

# Functions

In [2]:
toString_onefle.fx <- function(df, tostringvar){
    setDT(df)
    ab <- df[, .(tostringvar = toString( eval(parse( text = tostringvar))), #bind subjects
                 count = sum(count)),  #get sum of counts
             by = c("CDR3b", "TRBV", "TRBJ")] #get duplicates sequences with the same cdr3 + TRBV + TRBJ
    df_ab <- merge(df, ab, by = c("CDR3b","TRBV","TRBJ")) # merge together
    df_ab_dedup <- dplyr::distinct(df_ab, CDR3b, TRBV, TRBJ, tostringvar, .keep_all= TRUE)
    
    return(df_ab_dedup[, c("CDR3b", "TRBV", "TRBJ", "samplename", 
                           "count.y", "clonefraction", "tostringvar", "file")])
}

In [3]:
#source("/Users/anabbi/git/ped_CapTCRseq/R/Immune_diversity.R")

# Paths

In [1]:
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Plots/"
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Manifests/"

gitpath <- "/Users/anabbi/git/ped_CapTCRseq/"

In [2]:
h4hpath <- "/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/"

# Main

## Cleanup INT data

In [3]:
load(file = paste0(datapath,"capTCRseq/TRBfiles.RData"))

In [8]:
file_df_dedup_trb$patient <- substr(file_df_dedup_trb$samplename, 1,7)

In [9]:
head(file_df_dedup_trb)

,file_list,samplename,size,patient
,<chr>,<chr>,<dbl>,<chr>
264,batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,CHP_315-05-PBMC-DNA,16011,CHP_315
2,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-01-PBMC-DNA.txt,CHP_350-01-PBMC-DNA,85631,CHP_350
3,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-02-PBMC-DNA.txt,CHP_350-02-PBMC-DNA,143632,CHP_350
4,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-03-PBMC-DNA.txt,CHP_350-03-PBMC-DNA,107190,CHP_350
5,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-04-PBMC-DNA.txt,CHP_350-04-PBMC-DNA,54188,CHP_350
6,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_353-02-PBMC-DNA.txt,CHP_353-02-PBMC-DNA,9062,CHP_353


In [11]:
taball_list <- list()

for(p in unique(file_df_dedup_trb$patient)){
    
    mypatient <- file_df_dedup_trb[ file_df_dedup_trb$patient == p,]
    
    myplist <- list()
    for(i in 1:nrow(mypatient)){
        onefle <- read.table(paste0(h4hpath, mypatient$file_list[i]), sep = "\t", header = TRUE, stringsAsFactors = F)  
        # Remove non-productive
        onefle <- onefle[!grepl("_", onefle$aaSeqCDR3),]
        onefle <- onefle[!grepl("[*]", onefle$aaSeqCDR3),]
        onefle$CDR3b <- onefle$aaSeqCDR3
        #next if the file has no productive cdr3
        if(nrow(onefle) == 0){ next}
        #remove scores
        onefle$TRBV <- gsub("[*]0.*", "", onefle$allVHitsWithScore)
        onefle$TRBJ <- gsub("[*]0.*", "", onefle$allJHitsWithScore)
        #remove extra long cdr3s
        onefle <- onefle[ nchar(onefle$aaSeqCDR3) <= 25,]
        #next if the file has no productive cdr3
        if(nrow(onefle) == 0){ next}
        onefle$count <- onefle$cloneCount
        onefle$clonefraction <- onefle$cloneFraction
        onefle$file <- paste0(h4hpath, mypatient$file_list[i])
        onefle$samplename <- mypatient$samplename[i]
    
        onefle1 <- toString_onefle.fx(onefle, "nSeqCDR3")
    
        myplist[length(myplist) + 1] <- list(onefle1[,c("CDR3b", "TRBV", "TRBJ", "samplename", 
                                                     "count.y", "clonefraction","tostringvar", "file")])
    }
    gliph_input <- do.call("rbind", myplist)

    colnames(gliph_input)[ colnames(gliph_input) == "tostringvar"] <- "nSeqCDR3"
    colnames(gliph_input)[ colnames(gliph_input) == "count.y"] <- "count"

    #bind those with the same cdr3 + TRBV
    setDT(gliph_input)

    ab <- gliph_input[, .(subject = toString(samplename), #bind subjects
                          file = toString(file), #bind filenames
                          count = sum(count)),  #get sum of counts
                      by = c("CDR3b", "TRBV", "TRBJ")] #get duplicates sequences with the same cdr3 + TRBV + TRBJ

    gliph_input_ab <- merge(gliph_input, ab, by = c("CDR3b","TRBV","TRBJ")) # merge together

    #dedup
    gliph_input_ab_dedup <- dplyr::distinct(gliph_input_ab, CDR3b, TRBV, TRBJ, subject, .keep_all= TRUE)


    taball_list[length(taball_list) + 1] <- list(gliph_input_ab_dedup[,c("CDR3b", "TRBV", "TRBJ", "subject", 
                                                     "count.y", "clonefraction","file.y")])

    }

In [12]:
taball <- do.call("rbind", taball_list)

In [13]:
#some cleanup
taball$`subject:condition` <- substr(taball$subject, 1, 7)
colnames(taball)[ colnames(taball) == "count.y"] <- "count"
colnames(taball)[ colnames(taball) == "file.y"] <- "file"

In [14]:
head(taball)

CDR3b,TRBV,TRBJ,subject,count,clonefraction,file,subject:condition
<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>,<chr>
CAIRDPEQGITDTQYF,TRBV10-3,TRBJ2-3,CHP_315-05-PBMC-DNA,24,0.025369979,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,CHP_315
CAIRGTGADQPQHF,TRBV10-3,TRBJ1-5,CHP_315-05-PBMC-DNA,21,0.022198732,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,CHP_315
CAISGVWGGDTQYF,TRBV10-3,TRBJ2-3,CHP_315-05-PBMC-DNA,12,0.012684989,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,CHP_315
CASRLAGTGSPLHF,TRBV6-1,TRBJ1-6,CHP_315-05-PBMC-DNA,2,0.002114165,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,CHP_315
CASSDPQGTEAFF,TRBV6-2,TRBJ1-1,CHP_315-05-PBMC-DNA,21,0.022198732,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,CHP_315
CASSEEPGTSGYNEQFF,TRBV2,TRBJ2-1,CHP_315-05-PBMC-DNA,6,0.006342495,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,CHP_315


In [15]:
taball <- taball[,c(1,2,3,8,5,6,4,7)]

In [71]:
taball$index <- 1:nrow(taball)

In [72]:
save(taball, file = paste0(datapath, "capTCRseq/INT_gliph_input.RData"))

In [187]:
#save(taball, file = "/Users/anabbi/Desktop/H4H/INTERCEPT/Adaptive/tsv_cleanup/gliphinput/INT_gliph_input.RData")

## Cleanup VDJdb file - curated by Shirin

In [6]:
VDJdb_tcr <- read.table(file = paste0(datapath, "CapTCRseq/VDJdb_MinimalScoreConfidence3.tsv"), sep = "\t", header = T)

In [7]:
colnames(VDJdb_tcr)

[1] "complex.id"      "Gene"            "aaSeqCDR3"       "V"              
 [5] "J"               "MHC.A"           "MHC.B"           "MHC.class"      
 [9] "Epitope"         "Epitope.gene"    "Epitope.species" "Reference"

In [8]:
head(VDJdb_tcr)

,complex.id,Gene,aaSeqCDR3,V,J,MHC.A,MHC.B,MHC.class,Epitope,Epitope.gene,Epitope.species,Reference
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0,TRB,CAISGQAVSTDTQYF,TRBV10-3*01,TRBJ2-3*01,HLA-A*02,NA,MHCI,KLVALGINAV,NS3,HCV,PMID:27252176
2,0,TRB,CASDSNTEAFF,TRBV25-1*01,TRBJ1-1*01,HLA-A*02,NA,MHCI,KLVALGINAV,NS3,HCV,PMID:27252176
3,0,TRB,CASGWGSQPQHF,TRBV7-9*01,TRBJ1-5*01,HLA-A*11:01,NA,MHCI,AVFDRKSDAK,EBNA4,EBV,PMID:23267020
4,0,TRB,CASKGQGARDGYTF,TRBV6-6*01,TRBJ1-2*01,HLA-B*35:01,NA,MHCI,IPSINVHHY,pp65,CMV,PMID:23267020
5,0,TRB,CASKMRGDVYEQYF,TRBV18*01,TRBJ2-7*01,HLA-B*08:01,NA,MHCI,RAKFKQLL,BZLF1,EBV,PMID:23267020
6,0,TRB,CASMGNSAGANVLTF,TRBV10-1*01,TRBJ2-6*01,HLA-A*02,NA,MHCI,NLVPMVATV,pp65,CMV,PMID:9971792


In [9]:
VDJdb_tcr$`subject:condition` <- paste0("VDJdb:",VDJdb_tcr$Epitope.gene,"_", VDJdb_tcr$Epitope.species)
VDJdb_tcr$count <- 1
VDJdb_tcr$clonefraction <- 1
VDJdb_tcr$subject <- "VDJdb"
VDJdb_tcr$file <- "VDJdb_MinimalScoreConfidence3.tsv"
VDJdb_tcr$index <- 1:nrow(VDJdb_tcr)

In [10]:
VDJdb_tcr_cleanup <- VDJdb_tcr[, c("aaSeqCDR3", "V", "J", "subject:condition", "count", 
                                   "clonefraction", "subject", "file", "index")]

In [12]:
head(VDJdb_tcr_cleanup)

,aaSeqCDR3,V,J,subject:condition,count,clonefraction,subject,file,index
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<int>
1,CAISGQAVSTDTQYF,TRBV10-3*01,TRBJ2-3*01,VDJdb:NS3_HCV,1,1,VDJdb,VDJdb_MinimalScoreConfidence3.tsv,1
2,CASDSNTEAFF,TRBV25-1*01,TRBJ1-1*01,VDJdb:NS3_HCV,1,1,VDJdb,VDJdb_MinimalScoreConfidence3.tsv,2
3,CASGWGSQPQHF,TRBV7-9*01,TRBJ1-5*01,VDJdb:EBNA4_EBV,1,1,VDJdb,VDJdb_MinimalScoreConfidence3.tsv,3
4,CASKGQGARDGYTF,TRBV6-6*01,TRBJ1-2*01,VDJdb:pp65_CMV,1,1,VDJdb,VDJdb_MinimalScoreConfidence3.tsv,4
5,CASKMRGDVYEQYF,TRBV18*01,TRBJ2-7*01,VDJdb:BZLF1_EBV,1,1,VDJdb,VDJdb_MinimalScoreConfidence3.tsv,5
6,CASMGNSAGANVLTF,TRBV10-1*01,TRBJ2-6*01,VDJdb:pp65_CMV,1,1,VDJdb,VDJdb_MinimalScoreConfidence3.tsv,6


In [13]:
colnames(VDJdb_tcr_cleanup)[1:3] <- c('CDR3b', 'TRBV', 'TRBJ')

In [14]:
VDJdb_tcr_cleanup$TRBV <- gsub("[*]0.*", "", VDJdb_tcr_cleanup$TRBV)
VDJdb_tcr_cleanup$TRBJ <- gsub("[*]0.*", "", VDJdb_tcr_cleanup$TRBJ)

In [15]:
head(VDJdb_tcr_cleanup)

,CDR3b,TRBV,TRBJ,subject:condition,count,clonefraction,subject,file,index
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<int>
1,CAISGQAVSTDTQYF,TRBV10-3,TRBJ2-3,VDJdb:NS3_HCV,1,1,VDJdb,VDJdb_MinimalScoreConfidence3.tsv,1
2,CASDSNTEAFF,TRBV25-1,TRBJ1-1,VDJdb:NS3_HCV,1,1,VDJdb,VDJdb_MinimalScoreConfidence3.tsv,2
3,CASGWGSQPQHF,TRBV7-9,TRBJ1-5,VDJdb:EBNA4_EBV,1,1,VDJdb,VDJdb_MinimalScoreConfidence3.tsv,3
4,CASKGQGARDGYTF,TRBV6-6,TRBJ1-2,VDJdb:pp65_CMV,1,1,VDJdb,VDJdb_MinimalScoreConfidence3.tsv,4
5,CASKMRGDVYEQYF,TRBV18,TRBJ2-7,VDJdb:BZLF1_EBV,1,1,VDJdb,VDJdb_MinimalScoreConfidence3.tsv,5
6,CASMGNSAGANVLTF,TRBV10-1,TRBJ2-6,VDJdb:pp65_CMV,1,1,VDJdb,VDJdb_MinimalScoreConfidence3.tsv,6


In [16]:
save(VDJdb_tcr_cleanup, file = paste0(datapath, "capTCRseq/VDJdb_tcr_cleanup.RData"))

In [188]:
load(file = paste0(datapath, "capTCRseq/VDJdb_tcr_cleanup.RData"))

In [189]:
#save(VDJdb_tcr_cleanup, 
#     file = "/Users/anabbi/Desktop/H4H/INTERCEPT/Adaptive/tsv_cleanup/gliphinput/VDJdb_tcr_cleanup.RData")

## Cleanup Adaptive data

get a list of files

In [17]:
file_list <- list.files("/Users/anabbi/Desktop/H4H/projects/INTERCEPT/Adaptive/", 
                        pattern = ".tsv", full.names = T, recursive = T)

In [19]:
file_list <- file_list[ !grepl("SampleOverview", file_list)]

In [21]:
file_list1 <- list.files("/Users/anabbi/Desktop/H4H/pughlab/external_data/Adaptive/emerson2017", 
                        pattern = ".tsv", full.names = T, recursive = T)

In [22]:
file_list1 <- file_list1[ !grepl("SampleOverview", file_list1)]

In [23]:
allfiles <- c(file_list, file_list1)

take out files starting with Keck in emerson data and keep for validation

In [29]:
allfiles <- allfiles[ !grepl("Keck", allfiles)]
allfiles <- allfiles[ !grepl("Denver", allfiles)]

In [135]:
adaptivefiles <- allfiles

In [136]:
head(adaptivefiles)

[1] "/Users/anabbi/Desktop/H4H/projects/INTERCEPT/Adaptive//Carey_Preterm2017/Adult_1_15-04.tsv"
[2] "/Users/anabbi/Desktop/H4H/projects/INTERCEPT/Adaptive//Carey_Preterm2017/Adult_2_15-07.tsv"
[3] "/Users/anabbi/Desktop/H4H/projects/INTERCEPT/Adaptive//Carey_Preterm2017/Adult_3_15-03.tsv"
[4] "/Users/anabbi/Desktop/H4H/projects/INTERCEPT/Adaptive//Carey_Preterm2017/Adult_4_15-05.tsv"
[5] "/Users/anabbi/Desktop/H4H/projects/INTERCEPT/Adaptive//Carey_Preterm2017/Adult_5_15-06.tsv"
[6] "/Users/anabbi/Desktop/H4H/projects/INTERCEPT/Adaptive//Carey_Preterm2017/Cord_1_NWK206.tsv"

In [137]:
adaptivefiles <- gsub("/Users/anabbi/Desktop/H4H/projects/INTERCEPT/Adaptive//", 
                      "/cluster/projects/pughlab/projects/INTERCEPT/Adaptive/", adaptivefiles)

In [138]:
adaptivefiles <- gsub("/Users/anabbi/Desktop/H4H/pughlab/external_data/", 
                     "/cluster/projects/pughlab/external_data/", adaptivefiles)

In [139]:
write.table(adaptivefiles[grepl("emerson", adaptivefiles)], 
            file = "/Users/anabbi/Desktop/H4H/INTERCEPT/Adaptive/tsv_cleanup/emersonadaptivefiles.txt", 
            row.names = F, quote = F)

In [140]:
adaptivefiles[grepl("emers", adaptivefiles)]

[1] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00001.tsv"
  [2] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00002.tsv"
  [3] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00003.tsv"
  [4] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00004.tsv"
  [5] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00005.tsv"
  [6] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00006.tsv"
  [7] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00007.tsv"
  [8] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00008.tsv"
  [9] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00009.tsv"
 [10] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00010.tsv"
 [11] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00011.tsv"
 [12] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00012.tsv"
 [13] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00013.tsv"
 [14] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00014.tsv"
 [15] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00015.tsv"
 [16] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00016.tsv"
 [17] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00017.tsv"
 [18] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00018.tsv"
 [19] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00019.tsv"
 [20] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00020.tsv"
 [21] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00021.tsv"
 [22] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00022.tsv"
 [23] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00023.tsv"
 [24] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00024.tsv"
 [25] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00025.tsv"
 [26] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00026.tsv"
 [27] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00027.tsv"
 [28] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00028.tsv"
 [29] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00029.tsv"
 [30] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00030.tsv"
 [31] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00031.tsv"
 [32] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00032.tsv"
 [33] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00033.tsv"
 [34] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00034.tsv"
 [35] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00035.tsv"
 [36] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00036.tsv"
 [37] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00037.tsv"
 [38] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00038.tsv"
 [39] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00039.tsv"
 [40] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00040.tsv"
 [41] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00041.tsv"
 [42] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00042.tsv"
 [43] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00043.tsv"
 [44] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00044.tsv"
 [45] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00045.tsv"
 [46] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00046.tsv"
 [47] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00047.tsv"
 [48] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00048.tsv"
 [49] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00049.tsv"
 [50] "/cluster/projects/pughlab/external_data/Adaptive/emerson2017/P00050.tsv"
 [

In [92]:
#on h4h
clean_adaptive <- function(filename, outpath){
    
f1 <- read.table(filename, sep = "\t", header = T, stringsAsFactors = F)    

    #remove unproductive
f1 <- f1[f1$amino_acid != "",]
f1 <- f1[!grepl("[*]", f1$amino_acid),]
f1 <- f1[!grepl("_", f1$amino_acid),]
    
f1_clean <- f1[, c("amino_acid", "v_resolved", "j_resolved", "rearrangement", 
                   "seq_reads", "frequency", "sample_name")]
#add filename
f1_clean$file <- filename    
colnames(f1_clean) <- c("CDR3b", "TRBV", "TRBJ", "rearrangement", "clonecount", "clonefraction", "sample_id", "file")
    
    
#replaqce TCRB with TRB
f1_clean$TRBV <- gsub("TCRB", "TRB", f1_clean$TRBV)
f1_clean$TRBJ <- gsub("TCRB", "TRB", f1_clean$TRBJ)

f1_clean$TRBV <- gsub("[*].*", "", f1_clean$TRBV)
f1_clean$TRBJ <- gsub("[*].*", "", f1_clean$TRBJ)
    
write.table(f1_clean, paste0(outpath, f1_clean$sample_id[1]) , sep = "\t", quote = F, row.names = F)    
       
}

Run on h4h see "Adaptive/tsv_cleanup"

### compile all adaptive files together and cleanup

test here, run on h4h

In [142]:
myfilelist <- list.files("/Users/anabbi/Desktop/H4H/projects/INTERCEPT/Adaptive/tsv_cleanup/", 
                        pattern = ".txt", full.names = T, recursive = T)

In [155]:
myfilelist <- myfilelist[!grepl("files", myfilelist)]

In [161]:
head(myfilelist)

[1] "/Users/anabbi/Desktop/H4H/projects/INTERCEPT/Adaptive/tsv_cleanup//Carey_Preterm2017/Adult_1_15-04.txt"
[2] "/Users/anabbi/Desktop/H4H/projects/INTERCEPT/Adaptive/tsv_cleanup//Carey_Preterm2017/Adult_2_15-07.txt"
[3] "/Users/anabbi/Desktop/H4H/projects/INTERCEPT/Adaptive/tsv_cleanup//Carey_Preterm2017/Adult_3_15-03.txt"
[4] "/Users/anabbi/Desktop/H4H/projects/INTERCEPT/Adaptive/tsv_cleanup//Carey_Preterm2017/Adult_4_15-05.txt"
[5] "/Users/anabbi/Desktop/H4H/projects/INTERCEPT/Adaptive/tsv_cleanup//Carey_Preterm2017/Adult_5_15-06.txt"
[6] "/Users/anabbi/Desktop/H4H/projects/INTERCEPT/Adaptive/tsv_cleanup//Carey_Preterm2017/Cord_1_NWK206.txt"

In [157]:
mylst <- lapply(myfilelist[1:3], function(x){
    read.table(x, sep = "\t", header = T)
})

In [159]:
mydf <- do.call("rbind", mylst)

In [165]:
#cleanup
mydf$rearrangement <- NULL
mydf$`subject:condition` <- mydf$sample_id
mydf$subject <- mydf$sample_id
mydf$sample_id <- NULL
mydf$index <- 1:nrow(mydf)

In [171]:
#order
mydf <- mydf[,c("CDR3b", "TRBV", "TRBJ", "subject:condition", "clonecount", 
               "clonefraction", "subject", "file", "index")]

In [174]:
colnames(mydf)[ colnames(mydf) == "clonecount"] <- "count"

In [179]:
mydf$`subject:condition` <- paste(mydf$subject, ":", mydf$index, sep = "")

#save, then rbind with tabal and vdjdb, bind index with subject to create subject:condition, 


In [32]:
#bind those with the same cdr3 + TRBV + TRBJ
setDT(covidfle_uniq)

ab <- covidfle_uniq[, .(subject = toString(`subject:condition`), #bind subjects
                        count = sum(count)),  #get sum of counts
                    by = c("CDR3b", "TRBV", "TRBJ")] #get duplicates sequences with the same cdr3 + TRBV + TRBJ

covidfle_uniq_ab <- merge(covidfle_uniq, ab, by = c("CDR3b","TRBV","TRBJ")) # merge together

In [33]:
dim(covidfle_uniq_ab)

[1] 156033      7

In [34]:
colnames(covidfle_uniq_ab)

[1] "CDR3b"             "TRBV"              "TRBJ"             
[4] "subject:condition" "count.x"           "subject"          
[7] "count.y"

In [35]:
#dedup
covidfle_uniq_ab_dedup <- dplyr::distinct(covidfle_uniq_ab, 
                                          CDR3b, TRBV, TRBJ, subject, .keep_all= TRUE)

In [36]:
head(covidfle_uniq_ab_dedup)

CDR3b,TRBV,TRBJ,subject:condition,count.x,subject,count.y
<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>
AGVKITNSYEQYF,TCRBV20-X,TCRBJ02-07,eHO134,1,eHO134,1
AKNIQYF,TCRBV27-01,TCRBJ02-04,eQD126,1,eQD126,1
APYASKRAGYDNEQFF,TCRBV06-X,TCRBJ02-01,eHO141,1,eHO141,1
ARLFSGANVLTF,TCRBV07-02,TCRBJ02-06,eXL27,1,eXL27,1
CAAAAPDTQYF,TCRBV06-X,TCRBJ02-03,eOX46,1,eOX46,1
CAAADRMTDTQYF,TCRBV24-01,TCRBJ02-03,eHO130,1,eHO130,1


In [40]:
dim(covidfle_uniq_ab_dedup)

[1] 148678      7

In [38]:
#replaqce TCRB with TRB
covidfle_uniq_ab_dedup$TRBV <- gsub("TCRB", "TRB", covidfle_uniq_ab_dedup$TRBV)
covidfle_uniq_ab_dedup$TRBJ <- gsub("TCRB", "TRB", covidfle_uniq_ab_dedup$TRBJ)

In [41]:
head(covidfle_uniq_ab_dedup[order(covidfle_uniq_ab_dedup$count.y, decreasing = T),])

CDR3b,TRBV,TRBJ,subject:condition,count.x,subject,count.y
<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>
CASSIRSSYEQYF,TRBV19-01,TRBJ02-07,eOX46,1,"eOX46, ePD84, eHO128, eHO132, eHO125, ePD83, eHO136, eLH41, eHO124, ePD86, eLH42, eHO130, ePD76, eQD137, eQD123, eMR20, eEE243, eHH173, eHH174, eQD124, eHH169, eEE228, eXL27, eQD115, eXL36, eMR12, eLH47, eDH113, eEE226, eXL37, eDH96, eXL30",32
CASSQTGGRNQPQHF,TRBV12-X,TRBJ01-05,eLH41,1,"eLH41, eJL164, eXL31, eHO132, ePD86, eAV88, ePD84, eEE243, eXL37, eQD118, eEE224, eXL27, eMR12, eOX56, eLH47, eMR18, eLH46, eQD136, eOX43, eOX54, ePD76",21
CASSLAGAYEQYF,TRBV05-01,TRBJ02-07,eJL143,1,"eJL143, ePD79, eHH175, eLH45, eHO126, eXL31, eLH49, eLH42, eJL157, eHO135, eLH54, eMR14, eLH47, eAV91, eQD109, eQD111, eMR23, eQD123, eMR15",19
CASSFSYEQYF,TRBV12-03/12-04,TRBJ02-07,eEE243,1,"eEE243, eHH175, eXL30, eHO130, eJL149, eAV88, eXL27, eOX54, eEE224, eEE228, eOX49, eEE226, eOX52, eEE240, eOX46, eOX43, eXL31",17
CASSLSAPQETQYF,TRBV27-01,TRBJ02-05,eMR17,1,"eMR17, eJL161, eHO134, eQD124, eMR13, eQD111, eQD121, eHO140, eMR12, eQD125, eLH47, eEE226, eQD126, eQD114, eQD110, eQD120",16
CASSVGETQYF,TRBV09-01,TRBJ02-05,eOX56,1,"eOX56, eEE224, eEE226, eEE228, eOX49, eOX54, eOX52, eXL27, eXL30, eOX43, eOX46, eXL32, eEE240, eAV88, eQD128, eHH175",16


In [42]:
covidfle_uniq_ab_dedup$file <- "COVID_specific_TCRB.tsv"

In [45]:
covidfle_uniq_ab_dedup$clonefraction <- 1

In [47]:
covidfle_uniq_ab_dedup_subset <- covidfle_uniq_ab_dedup[,c("CDR3b", "TRBV", "TRBJ", "subject:condition",
                                                          "count.y", "clonefraction", "subject","file")]

In [49]:
colnames(covidfle_uniq_ab_dedup_subset)[ colnames(covidfle_uniq_ab_dedup_subset) == "count.y"] <- "count"

In [73]:
covidfle_uniq_ab_dedup_subset$index <- 1:nrow(covidfle_uniq_ab_dedup_subset)

In [74]:
save(covidfle_uniq_ab_dedup_subset, file = paste0(datapath, "capTCRseq/covidfle_uniq_ab_dedup_subset.RData"))

# bind all

In [162]:
load(file = paste0(datapath, "capTCRseq/INT_gliph_input.RData"))

In [163]:
head(taball)

CDR3b,TRBV,TRBJ,subject:condition,count,clonefraction,subject,file,index
<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>,<chr>,<int>
CAIRDPEQGITDTQYF,TRBV10-3,TRBJ2-3,CHP_315,24,0.025369979,CHP_315-05-PBMC-DNA,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,1
CAIRGTGADQPQHF,TRBV10-3,TRBJ1-5,CHP_315,21,0.022198732,CHP_315-05-PBMC-DNA,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,2
CAISGVWGGDTQYF,TRBV10-3,TRBJ2-3,CHP_315,12,0.012684989,CHP_315-05-PBMC-DNA,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,3
CASRLAGTGSPLHF,TRBV6-1,TRBJ1-6,CHP_315,2,0.002114165,CHP_315-05-PBMC-DNA,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,4
CASSDPQGTEAFF,TRBV6-2,TRBJ1-1,CHP_315,21,0.022198732,CHP_315-05-PBMC-DNA,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,5
CASSEEPGTSGYNEQFF,TRBV2,TRBJ2-1,CHP_315,6,0.006342495,CHP_315-05-PBMC-DNA,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,6


In [7]:
#load(file = paste0(datapath, "capTCRseq/covidfle_uniq_ab_dedup_subset.RData"))

In [8]:
load(file = paste0(datapath, "capTCRseq/VDJdb_tcr_cleanup.RData"))

In [9]:
gliph_int_covid_tcr <- rbind(taball, covidfle_uniq_ab_dedup_subset, VDJdb_tcr_cleanup)

In [10]:
gliph_int_covid_tcr$CDR3a <- NA

In [11]:
gliph_int_covid_tcr <- gliph_int_covid_tcr[,c('CDR3b','TRBV','TRBJ','CDR3a', 'subject:condition','count',
                                              'clonefraction','subject','file','index')]

In [12]:
#bind subject and index to be able to map back to gliph_int_covid_tcr
gliph_int_covid_tcr$`subject:condition` <- paste(gliph_int_covid_tcr$`subject:condition`,
                                                 gliph_int_covid_tcr$index, sep = ":")

In [13]:
save(gliph_int_covid_tcr, file = paste0(datapath, "capTCRseq/gliph_int_covid_tcr.RData"))

In [14]:
#remove header and select cols 1:6 for gliph to work
write.table(gliph_int_covid_tcr[,1:6], file = paste0(h4hpath,'/GLIPH/gliph_int_covid_tcr.txt'),
         sep = "\t", quote = F, row.names = F, col.names = F)

In [182]:
load(file = paste0(datapath, "capTCRseq/gliph_int_covid_tcr.RData"))

In [183]:
head(gliph_int_covid_tcr)

CDR3b,TRBV,TRBJ,CDR3a,subject:condition,count,clonefraction,subject,file,index
<chr>,<chr>,<chr>,<lgl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<int>
CAIRDPEQGITDTQYF,TRBV10-3,TRBJ2-3,NA,CHP_315:1,24,0.025369979,CHP_315-05-PBMC-DNA,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,1
CAIRGTGADQPQHF,TRBV10-3,TRBJ1-5,NA,CHP_315:2,21,0.022198732,CHP_315-05-PBMC-DNA,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,2
CAISGVWGGDTQYF,TRBV10-3,TRBJ2-3,NA,CHP_315:3,12,0.012684989,CHP_315-05-PBMC-DNA,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,3
CASRLAGTGSPLHF,TRBV6-1,TRBJ1-6,NA,CHP_315:4,2,0.002114165,CHP_315-05-PBMC-DNA,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,4
CASSDPQGTEAFF,TRBV6-2,TRBJ1-1,NA,CHP_315:5,21,0.022198732,CHP_315-05-PBMC-DNA,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,5
CASSEEPGTSGYNEQFF,TRBV2,TRBJ2-1,NA,CHP_315:6,6,0.006342495,CHP_315-05-PBMC-DNA,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,6


# Backup

In [6]:
mylist <- list()

for(i in 1:nrow(file_df_dedup_trb)){
    
    onefle <- read.table(paste0(h4hpath, file_df_dedup_trb$file_list[i]), 
                         sep = "\t", header = TRUE, stringsAsFactors = F)  
    # Remove non-productive
    onefle <- onefle[!grepl("_", onefle$aaSeqCDR3),]
    onefle <- onefle[!grepl("[*]", onefle$aaSeqCDR3),]
    onefle$CDR3b <- onefle$aaSeqCDR3
    #next if the file has no productive cdr3
    if(nrow(onefle) == 0){ next}
    #remove scores
    onefle$TRBV <- gsub("[*]0.*", "", onefle$allVHitsWithScore)
    onefle$TRBJ <- gsub("[*]0.*", "", onefle$allJHitsWithScore)

    onefle$count <- onefle$cloneCount
    onefle$clonefraction <- onefle$cloneFraction
    onefle$nSeqCDR3 <- onefle$nSeqCDR3
    onefle$file <- paste0(h4hpath, file_df_dedup_trb$file_list[i])
    onefle$subject.condition <- file_df_dedup_trb$samplename[i]
    mylist[length(mylist) + 1] <- list(onefle[,c("CDR3b", "TRBV", "TRBJ", "subject.condition", 
                                                 "count", "clonefraction","nSeqCDR3", "file")])
}

In [7]:
length(mylist)

[1] 580

In [8]:
gliph_input <- do.call("rbind", mylist)

In [9]:
dim(gliph_input)

[1] 72634     8

In [10]:
head(gliph_input)

,CDR3b,TRBV,TRBJ,subject.condition,count,clonefraction,nSeqCDR3,file
,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>,<chr>
1,CASSFYGREADTQYF,TRBV7-2,TRBJ2-3,CHP_315-05-PBMC-DNA,187,0.19767442,TGTGCCAGCAGCTTCTATGGCAGGGAAGCAGATACGCAGTATTTT,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt
3,CASSLMGGDSTEAFF,TRBV7-3,TRBJ1-1,CHP_315-05-PBMC-DNA,48,0.05073996,TGTGCCAGCAGCTTAATGGGGGGAGATAGCACTGAAGCTTTCTTT,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt
4,CASSLTGTGVEKLFF,TRBV7-3,TRBJ1-4,CHP_315-05-PBMC-DNA,48,0.05073996,TGTGCCAGCAGCTTAACCGGGACAGGGGTTGAAAAACTGTTTTTT,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt
5,CASSQRGQSTEQYF,TRBV14,TRBJ2-7,CHP_315-05-PBMC-DNA,42,0.04439746,TGTGCCAGCAGCCAACGGGGACAGTCAACCGAGCAGTACTTC,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt
6,CASSYQAGVGETQYF,TRBV6-6,TRBJ2-5,CHP_315-05-PBMC-DNA,32,0.03382664,TGTGCCAGCAGTTACCAAGCGGGGGTGGGTGAGACCCAGTACTTC,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt
9,CASSVVGVYNSPLHF,TRBV7-2,TRBJ1-6,CHP_315-05-PBMC-DNA,29,0.03065539,TGTGCCAGCAGCGTAGTAGGGGTCTATAATTCACCCCTCCACTTT,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt


In [12]:
library(data.table)

#bind those with the same cdr3 + TRBV
setDT(gliph_input)

ab <- gliph_input[, .(subject = toString(subject.condition), #bind subjects
                      file = toString(file), #bind filenames
                      count = sum(count)),  #get sum of counts
                  by = c("CDR3b", "TRBV", "TRBJ")] #get duplicates sequences with the same cdr3 + TRBV + TRBJ

gliph_input_ab <- merge(gliph_input, ab, by = c("CDR3b","TRBV","TRBJ")) # merge together

In [13]:
head(gliph_input_ab)

CDR3b,TRBV,TRBJ,subject.condition,count.x,clonefraction,nSeqCDR3,file.x,subject,file.y,count.y
<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>
ADITWRGGQPQHF,TRBV3-2,TRBJ1-5,CHP_387-02_PBMC_DNA,12,0.0017035775,GCCGACATTACCTGGAGAGGGGGTCAGCCCCAGCATTTT,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch11_23Apr2020/mixcr/clones/CLONES_TRBCHP_387-02_PBMC_DNA.txt,CHP_387-02_PBMC_DNA,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch11_23Apr2020/mixcr/clones/CLONES_TRBCHP_387-02_PBMC_DNA.txt,12
CAAAGGGEAFF,TRBV28,TRBJ1-1,CHP_396-03_PBMC_DNA,16,0.0033105731,TGTGCCGCAGCGGGGGGAGGTGAAGCTTTCTTT,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch11_23Apr2020/mixcr/clones/CLONES_TRBCHP_396-03_PBMC_DNA.txt,"CHP_396-03_PBMC_DNA, CHP_396-04-PBMC-DNA","/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch11_23Apr2020/mixcr/clones/CLONES_TRBCHP_396-03_PBMC_DNA.txt, /Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch13_26Aug2020/mixcr/clones/CLONES_TRBCHP_396-04-PBMC-DNA.txt",340
CAAAGGGEAFF,TRBV28,TRBJ1-1,CHP_396-04-PBMC-DNA,324,0.0012697069,TGTGCCGCAGCGGGGGGAGGTGAAGCTTTCTTT,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch13_26Aug2020/mixcr/clones/CLONES_TRBCHP_396-04-PBMC-DNA.txt,"CHP_396-03_PBMC_DNA, CHP_396-04-PBMC-DNA","/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch11_23Apr2020/mixcr/clones/CLONES_TRBCHP_396-03_PBMC_DNA.txt, /Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch13_26Aug2020/mixcr/clones/CLONES_TRBCHP_396-04-PBMC-DNA.txt",340
CAAAGGSGVGYEQYF,TRBV3-1,TRBJ2-7,CHP_393-05-PBMC-DNA,151,0.0012674058,TGTGCCGCCGCCGGGGGCAGCGGGGTGGGCTACGAGCAGTACTTC,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch13_26Aug2020/mixcr/clones/CLONES_TRBCHP_393-05-PBMC-DNA.txt,CHP_393-05-PBMC-DNA,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch13_26Aug2020/mixcr/clones/CLONES_TRBCHP_393-05-PBMC-DNA.txt,151
CAAARGFDEQYF,TRBV10-3,TRBJ2-7,CHP-411-02-PBMC-DNA,20,0.0001310135,TGTGCCGCCGCACGGGGTTTCGACGAGCAGTACTTC,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/remaining_batch_4_28June2021/mixcr/clones/CLONES_TRBCHP-411-02-PBMC-DNA.txt,CHP-411-02-PBMC-DNA,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/remaining_batch_4_28June2021/mixcr/clones/CLONES_TRBCHP-411-02-PBMC-DNA.txt,20
CAACEPEVRDTDTQYF,TRBV28,TRBJ2-3,CHP_338-05-PBMC-DNA,11,0.0026397888,TGTGCCGCCTGCGAACCTGAAGTACGGGACACAGATACGCAGTATTTT,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch2_Aug62019/mixcr/clones/CLONES_TRBCHP_338-05-PBMC-DNA.txt,"CHP_338-05-PBMC-DNA, CHP_338-05-PBMC-DNA","/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch2_Aug62019/mixcr/clones/CLONES_TRBCHP_338-05-PBMC-DNA.txt, /Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch2_Aug62019/mixcr/clones/CLONES_TRBCHP_338-05-PBMC-DNA.txt",12


In [14]:
#dedup
gliph_input_ab_dedup <- dplyr::distinct(gliph_input_ab, CDR3b, TRBV, TRBJ, subject, .keep_all= TRUE)

In [15]:
dim(gliph_input_ab)
dim(gliph_input_ab_dedup)

[1] 72634    11

[1] 68956    11

In [16]:
datapath

[1] "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Data/"

In [17]:
#index column incase I want to map gliph_input_dedup_ab_subset back to gliph_input_dedup_ab
gliph_input_ab_dedup$index <- 1:nrow(gliph_input_ab_dedup)

In [18]:
save(gliph_input_ab_dedup, file = paste0(datapath, "capTCRseq/gliph_input_ab_dedup.RData"))

In [19]:
colnames(gliph_input_ab_dedup)

[1] "CDR3b"             "TRBV"              "TRBJ"             
 [4] "subject.condition" "count.x"           "clonefraction"    
 [7] "nSeqCDR3"          "file.x"            "subject"          
[10] "file.y"            "count.y"           "index"

In [20]:
#reorder and subset to input to gliph
gliph_input_ab_dedup_subset <- gliph_input_ab_dedup[, c("CDR3b", "TRBV", "TRBJ", "subject", "count.y", "file.y", "index")]
                                                         

In [21]:
colnames(gliph_input_ab_dedup_subset) <- c("CDR3b", "TRBV", "TRBJ", "subject:condition", "count", "file", "index")

In [22]:
save(gliph_input_ab_dedup_subset, file = paste0(datapath, "capTCRseq/gliph_input_ab_dedup_subset.RData"))

## Cleanup Adaptive data

In [6]:
emerson <- read.table('/Users/anabbi/Desktop/H4H/pughlab/external_data/Adaptive/emerson2017/P00496.tsv', sep = "\t", header = F)

In [7]:
head(emerson)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,sample_name,species,locus,product_subtype,kit_pool,sku,test_name,sample_catalog_tags,sample_rich_tags,sample_rich_tags_json,⋯,cdr3_rearrangement,cdr3_amino_acid,cdr3_start_index,cdr3_rearrangement_length,chosen_v_family,chosen_v_gene,chosen_v_allele,chosen_j_family,chosen_j_gene,chosen_j_allele
2,P00496,Human,TCRB,UltraDeep,,,,"Age (Range):35-60 Years,Age (Range):45-65 Years (Late Adulthood),Age (Range):50-54 Years,Age:50 Years,Biological Sex:Female,Ethnic Group:Non-Hispanic or Latino,HLA MHC class I:HLA-A*02,HLA MHC class I:HLA-A*34,HLA MHC class I:HLA-B*15,HLA MHC class I:HLA-B*38,Racial Group:Asian or Pacific Islander,Tissue Source:PBMC,Tissue Source:Peripheral blood lymphocytes (PBL),Tissue Source:T cells,Tissue Source:gDNA,Virus Diseases:Cytomegalovirus +",null,,⋯,TGTGCCAGCAGCCTGACTCTAAACACTGAAGCTTTCTTT,CASSLTLNTEAFF,327,39,TCRBV05,05,01,TCRBJ01,01,01
3,P00496,Human,TCRB,UltraDeep,,,,"Age (Range):35-60 Years,Age (Range):45-65 Years (Late Adulthood),Age (Range):50-54 Years,Age:50 Years,Biological Sex:Female,Ethnic Group:Non-Hispanic or Latino,HLA MHC class I:HLA-A*02,HLA MHC class I:HLA-A*34,HLA MHC class I:HLA-B*15,HLA MHC class I:HLA-B*38,Racial Group:Asian or Pacific Islander,Tissue Source:PBMC,Tissue Source:Peripheral blood lymphocytes (PBL),Tissue Source:T cells,Tissue Source:gDNA,Virus Diseases:Cytomegalovirus +",null,,⋯,TGTGCCAGCAGCTTGGAACTCAATCAGCCCCAGCATTTT,CASSLELNQPQHF,327,39,TCRBV05,05,01,TCRBJ01,05,01
4,P00496,Human,TCRB,UltraDeep,,,,"Age (Range):35-60 Years,Age (Range):45-65 Years (Late Adulthood),Age (Range):50-54 Years,Age:50 Years,Biological Sex:Female,Ethnic Group:Non-Hispanic or Latino,HLA MHC class I:HLA-A*02,HLA MHC class I:HLA-A*34,HLA MHC class I:HLA-B*15,HLA MHC class I:HLA-B*38,Racial Group:Asian or Pacific Islander,Tissue Source:PBMC,Tissue Source:Peripheral blood lymphocytes (PBL),Tissue Source:T cells,Tissue Source:gDNA,Virus Diseases:Cytomegalovirus +",null,,⋯,TGTGCCAGCGCCCGGGGGAACACTGAAGCTTTCTTT,CASARGNTEAFF,327,36,TCRBV05,05,01,TCRBJ01,01,01
5,P00496,Human,TCRB,UltraDeep,,,,"Age (Range):35-60 Years,Age (Range):45-65 Years (Late Adulthood),Age (Range):50-54 Years,Age:50 Years,Biological Sex:Female,Ethnic Group:Non-Hispanic or Latino,HLA MHC class I:HLA-A*02,HLA MHC class I:HLA-A*34,HLA MHC class I:HLA-B*15,HLA MHC class I:HLA-B*38,Racial Group:Asian or Pacific Islander,Tissue Source:PBMC,Tissue Source:Peripheral blood lymphocytes (PBL),Tissue Source:T cells,Tissue Source:gDNA,Virus Diseases:Cytomegalovirus +",null,,⋯,,,,0,,,,,,
6,P00496,Human,TCRB,UltraDeep,,,,"Age (Range):35-60 Years,Age (Range):45-65 Years (Late Adulthood),Age (Range):50-54 Years,Age:50 Years,Biological Sex:Female,Ethnic Group:Non-Hispanic or Latino,HLA MHC class I:HLA-A*02,HLA MHC class I:HLA-A*34,HLA MHC class I:HLA-B*15,HLA MHC class I:HLA-B*38,Racial Group:Asian or Pacific Islander,Tissue Source:PBMC,Tissue Source:Peripheral blood lymphocytes (PBL),Tissue Source:T cells,Tissue Source:gDNA,Virus Diseases:Cytomegalovirus +",null,,⋯,TGTGCCAGCAGACCCAAGAGCTCCTACAATGAGCAGTTCTTC,CASRPKSSYNEQFF,327,42,TCRBV05,05,01,TCRBJ02,01,01


In [23]:
dim(covidfle)

[1] 162651      6

In [24]:
#remove unproductive
covidfle <- covidfle[!grepl("[*]", covidfle$V1),]
covidfle <- covidfle[ covidfle$V1 != "unproductive",]

In [25]:
dim(covidfle)

[1] 160130      6

In [26]:
#remove duplicated entries
covidfle_uniq <- unique(covidfle)

In [27]:
dim(covidfle_uniq)

[1] 156033      6

In [28]:
covidfle_uniq$V4 <- NULL

In [29]:
head(covidfle_uniq)

,V1,V2,V3,V5,V6
,<chr>,<chr>,<chr>,<chr>,<int>
1,CASGRHNSPLHF,TCRBV12-03/12-04,TCRBJ01-06,eJL143,1
2,CASRPAGVRNEQFF,TCRBV06-06,TCRBJ02-01,eJL146,1
3,CSASPRTLWGEQFF,TCRBV20-X,TCRBJ02-01,eJL146,1
4,CASSLVTSVKDTQYF,TCRBV05-01,TCRBJ02-03,eJL146,1
5,CASSAIVAVYGYTF,TCRBV07-08,TCRBJ01-02,eJL146,1
6,CSVYSAGELFF,TCRBV29-01,TCRBJ02-02,eJL146,1


In [30]:
colnames(covidfle_uniq) <- c("CDR3b", "TRBV", "TRBJ", "subject:condition", "count")

In [31]:
colnames(covidfle_uniq)

[1] "CDR3b"             "TRBV"              "TRBJ"             
[4] "subject:condition" "count"

In [32]:
#bind those with the same cdr3 + TRBV + TRBJ
setDT(covidfle_uniq)

ab <- covidfle_uniq[, .(subject = toString(`subject:condition`), #bind subjects
                        count = sum(count)),  #get sum of counts
                    by = c("CDR3b", "TRBV", "TRBJ")] #get duplicates sequences with the same cdr3 + TRBV + TRBJ

covidfle_uniq_ab <- merge(covidfle_uniq, ab, by = c("CDR3b","TRBV","TRBJ")) # merge together

In [33]:
dim(covidfle_uniq_ab)

[1] 156033      7

In [34]:
colnames(covidfle_uniq_ab)

[1] "CDR3b"             "TRBV"              "TRBJ"             
[4] "subject:condition" "count.x"           "subject"          
[7] "count.y"

In [35]:
#dedup
covidfle_uniq_ab_dedup <- dplyr::distinct(covidfle_uniq_ab, 
                                          CDR3b, TRBV, TRBJ, subject, .keep_all= TRUE)

In [36]:
head(covidfle_uniq_ab_dedup)

CDR3b,TRBV,TRBJ,subject:condition,count.x,subject,count.y
<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>
AGVKITNSYEQYF,TCRBV20-X,TCRBJ02-07,eHO134,1,eHO134,1
AKNIQYF,TCRBV27-01,TCRBJ02-04,eQD126,1,eQD126,1
APYASKRAGYDNEQFF,TCRBV06-X,TCRBJ02-01,eHO141,1,eHO141,1
ARLFSGANVLTF,TCRBV07-02,TCRBJ02-06,eXL27,1,eXL27,1
CAAAAPDTQYF,TCRBV06-X,TCRBJ02-03,eOX46,1,eOX46,1
CAAADRMTDTQYF,TCRBV24-01,TCRBJ02-03,eHO130,1,eHO130,1


In [40]:
dim(covidfle_uniq_ab_dedup)

[1] 148678      7

In [38]:
#replaqce TCRB with TRB
covidfle_uniq_ab_dedup$TRBV <- gsub("TCRB", "TRB", covidfle_uniq_ab_dedup$TRBV)
covidfle_uniq_ab_dedup$TRBJ <- gsub("TCRB", "TRB", covidfle_uniq_ab_dedup$TRBJ)

In [41]:
head(covidfle_uniq_ab_dedup[order(covidfle_uniq_ab_dedup$count.y, decreasing = T),])

CDR3b,TRBV,TRBJ,subject:condition,count.x,subject,count.y
<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>
CASSIRSSYEQYF,TRBV19-01,TRBJ02-07,eOX46,1,"eOX46, ePD84, eHO128, eHO132, eHO125, ePD83, eHO136, eLH41, eHO124, ePD86, eLH42, eHO130, ePD76, eQD137, eQD123, eMR20, eEE243, eHH173, eHH174, eQD124, eHH169, eEE228, eXL27, eQD115, eXL36, eMR12, eLH47, eDH113, eEE226, eXL37, eDH96, eXL30",32
CASSQTGGRNQPQHF,TRBV12-X,TRBJ01-05,eLH41,1,"eLH41, eJL164, eXL31, eHO132, ePD86, eAV88, ePD84, eEE243, eXL37, eQD118, eEE224, eXL27, eMR12, eOX56, eLH47, eMR18, eLH46, eQD136, eOX43, eOX54, ePD76",21
CASSLAGAYEQYF,TRBV05-01,TRBJ02-07,eJL143,1,"eJL143, ePD79, eHH175, eLH45, eHO126, eXL31, eLH49, eLH42, eJL157, eHO135, eLH54, eMR14, eLH47, eAV91, eQD109, eQD111, eMR23, eQD123, eMR15",19
CASSFSYEQYF,TRBV12-03/12-04,TRBJ02-07,eEE243,1,"eEE243, eHH175, eXL30, eHO130, eJL149, eAV88, eXL27, eOX54, eEE224, eEE228, eOX49, eEE226, eOX52, eEE240, eOX46, eOX43, eXL31",17
CASSLSAPQETQYF,TRBV27-01,TRBJ02-05,eMR17,1,"eMR17, eJL161, eHO134, eQD124, eMR13, eQD111, eQD121, eHO140, eMR12, eQD125, eLH47, eEE226, eQD126, eQD114, eQD110, eQD120",16
CASSVGETQYF,TRBV09-01,TRBJ02-05,eOX56,1,"eOX56, eEE224, eEE226, eEE228, eOX49, eOX54, eOX52, eXL27, eXL30, eOX43, eOX46, eXL32, eEE240, eAV88, eQD128, eHH175",16


In [42]:
covidfle_uniq_ab_dedup$file <- "COVID_specific_TCRB.tsv"

In [45]:
covidfle_uniq_ab_dedup$clonefraction <- 1

In [46]:
colnames(covidfle_uniq_ab_dedup)

[1] "CDR3b"             "TRBV"              "TRBJ"             
[4] "subject:condition" "count.x"           "subject"          
[7] "count.y"           "file"              "clonefraction"

In [47]:
covidfle_uniq_ab_dedup_subset <- covidfle_uniq_ab_dedup[,c("CDR3b", "TRBV", "TRBJ", "subject:condition",
                                                          "count.y", "clonefraction", "subject","file")]

In [49]:
colnames(covidfle_uniq_ab_dedup_subset)[ colnames(covidfle_uniq_ab_dedup_subset) == "count.y"] <- "count"

In [73]:
covidfle_uniq_ab_dedup_subset$index <- 1:nrow(covidfle_uniq_ab_dedup_subset)

In [74]:
save(covidfle_uniq_ab_dedup_subset, file = paste0(datapath, "capTCRseq/covidfle_uniq_ab_dedup_subset.RData"))

In [65]:
#write.table(covidfle_uniq_ab_dedup_subset, file = '/Users/anabbi/Desktop/H4H/projects/COVID-19/tcr-cap/Adaptive/MIRA/ImmuneCODE-MIRA-Release002.1/COVID_specific_TCRB_clean.tsv',
#            sep = "\t", quote = F, row.names = F)